# BIO-SELECT - Marigliano
## Features merging using several lists

_TODO_ : insert global pipeline image here + highlight this notebook on the picture

## Imports

In [ ]:
from sklearn import neighbors, datasets
import pandas as pd
import os
from matplotlib import pyplot as plt
import numpy as np
from sklearn import preprocessing

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC

from utils.ConfusionMatrix import ConfusionMatrix

import itertools
from sklearn.metrics import confusion_matrix

import math

%matplotlib inline

# set float precision at 2 digits
np.set_printoptions(precision=2)

# set the random seed for reproducibility
#np.random.seed(4)

In [ ]:
# Use Golub
GROUP_NAME = "golub_19122016"
DATASET = "Golub" # choose between "Golub" and "MILE"

# Use MILE
#GROUP_NAME = "mile_19122016"
#DATASET = "MILE"

## Load the features lists

TODO: load the features lists from CSV files

In [ ]:
from utils.CSVFeaturesImporter import CSVFeaturesImporter

importer = CSVFeaturesImporter(GROUP_NAME)
subsets = importer.load()
#print(subsets["features"].keys())
#print(subsets["features_by_score"]["ReliefF"][:5])


features = subsets["features"]

## Features subsets merging
Each algorithm has done its work and provide a subset of features as:
* a ranked score list
* a ranked list (no score)
* a list (no ranking, no score)

This part uses some techniques to combine/merge theses lists into a better one

_TODO_: 
* Visualize the lists
    * Venn diagram ? --> limited to 3 sets, does not scale
    * matrix: show the similarity of features between two subsets
        * Jaccard
        * Union
* implement merge techniques
    * votation
    * weighted votation
    * union of intersection
    * ...

### Subsets visualization

In [ ]:
from scipy import spatial

# some set similarity functions
def intersection_count(a, b):
    return len(a.intersection(b))

def jaccard(a, b):
    return len(a.intersection(b))/float(len(a.union(b)))

def compute_similary_between_subsets(subsets, compare_func):
    N_subsets = len(subsets)
    similarity_matrix = np.zeros(shape=(N_subsets, N_subsets))

    for i, j in itertools.product(range(N_subsets), range(N_subsets)):
        if isinstance(subsets[0][0], int):
            subset_i = set(subsets[i])
            subset_j = set(subsets[j])
        else:
            subset_i = {i[0] for i in subsets[i]}
            subset_j = {j[0] for j in subsets[j]}

        similarity_matrix[i, j] = compare_func(subset_i, subset_j)
        
    return similarity_matrix

def plot_feature_subsets_matrix(cm, alg_names, title, cmap=plt.cm.Blues):
    title += "\n" # add a little margin below the title
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    #plt.colorbar()
    plt.title(title)
    tick_marks = np.arange(len(alg_names))
    plt.xticks(tick_marks, alg_names, rotation=45)
    plt.yticks(tick_marks, alg_names)

    thresh = cm.max() / 2.0 + 0.1
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        text = "%.2f" % cm[i, j]
        plt.text(j, i, text,
                 horizontalalignment="center",
                 backgroundcolor="white",
                 #color="white" if cm[i, j] > thresh else "black")
                 color="black")

    plt.tight_layout()

# plot the similarity matrices
alg_names, features_subsets = zip(*features.items())

plt.figure(figsize=(12, 12))

similarity_matrix = compute_similary_between_subsets(features_subsets, compare_func=jaccard)
plt.subplot(1,2,1)
plot_feature_subsets_matrix(similarity_matrix, alg_names, title="Jaccard similarity between two feature subsets")

similarity_matrix = compute_similary_between_subsets(features_subsets, compare_func=intersection_count)
plt.subplot(1,2,2)
plot_feature_subsets_matrix(similarity_matrix, alg_names, title="Intersection between two feature subsets")

#### Dendrogram - visualizing the "distance" between the lists

In [ ]:
f_names, f_values = zip(*subsets["features"].items())

# only keep the features indices, drop the features occurences
def extract_lists(f_values):
    for fv in f_values:
        try:
            yield [f_idx for f_idx, _ in fv]
        except ValueError:
            pass
            
            
f_values = [i for i in extract_lists(f_values)]

In [ ]:
import numpy as np
from scipy.cluster.hierarchy import linkage
from matplotlib import pyplot as plt
from scipy.cluster.hierarchy import dendrogram

def get_mask_of_features(a, union):
    return [u_i in a for u_i in union]

def fancy_dendrogram(*args, **kwargs):
    """
    Source: https://joernhees.de/blog/2015/08/26/scipy-hierarchical-clustering-and-dendrogram-tutorial/
    """
    max_d = kwargs.pop('max_d', None)
    if max_d and 'color_threshold' not in kwargs:
        kwargs['color_threshold'] = max_d

    ddata = dendrogram(*args, **kwargs)

    if not kwargs.get('no_plot', False):
        for i, d, c in zip(ddata['icoord'], ddata['dcoord'], ddata['color_list']):
            x = 0.5 * sum(i[1:3])
            y = d[1]
            plt.plot(x, y, 'o', c=c)
            plt.annotate("%.3g" % y, (x, y), xytext=(0, -5),
                         textcoords='offset points',
                         va='top', ha='center')
        if max_d:
            plt.axhline(y=max_d, c='k')
    return ddata


L = f_values
L_labels = f_names

# Use the union of all the selected features in all the lists to create the mask.
# Using the union allows us to reduce the size of the mask by ignoring the features that are not in any lists
# The mask is simply a list of booleans. True if the feature is in the list, False otherwise
# Warning: we lose the sorting with the union
u = list(reduce(lambda a, b: set(a).union(set(b)), L))

L_mask = [get_mask_of_features(f, u) for f in L]
L_mask = np.array(L_mask)


#Z = linkage(L_mask, metric='rogerstanimoto')
#Z = linkage(L_mask, metric='jaccard')

metrics = [
    'rogerstanimoto',
    'jaccard',
    'dice',
    'russellrao',
    'yule'
]
for m in metrics:
    Z = linkage(L_mask, metric=m)
    # calculate full dendrogram
    plt.figure(figsize=(12, 4))
    plt.title('Hierarchical Clustering Dendrogram\n' + m)
    plt.xlabel('lists of selected features')
    plt.ylabel('distance')

    fancy_dendrogram(
        Z,
        labels=L_labels,
        leaf_rotation=90.,  # rotates the x axis labels,
    )
    plt.show()

We can see that the lists of F Value and Fisher Score are the same (like the similarity matrix has shown).

__For Golub only:__

All the features in CFS are in MRMR (see the intersection in the similarity matrix). But CFS only contains 9 features in total. So the mask of features for CFS is almost a list of False values which means that the distance to the other lists (including MRMR) is high.

### Subsets merging

In [ ]:
# ensure that when we merge the lists of features, the list remains composed of unique features
def assert_list_contains_only_unique_features(features):
    assert len(features) == len(set(features))


In [ ]:
merged_features_lists = []

In [ ]:
from merge.simple.SimpleUnionSubsetMerger import SimpleUnionSubsetMerger

susm = SimpleUnionSubsetMerger(features_subsets)
merged_features = susm.merge()
n_all_features = sum([len(s) for s in features_subsets])

print("Unique features (union of all subsets): %d over a total of %d " % (len(merged_features), n_all_features))

merged_features_lists.append(("Union of all features", merged_features, -1))

In [ ]:
subsets.keys()

In [ ]:
def group_by_features(features):
    from itertools import groupby
    
    def keyfunc(x): return x[0]
    
    list_of_lists_sorted = sorted(features, key=keyfunc)
    grouped_list = [list(j) for i, j in groupby(list_of_lists_sorted, key=keyfunc)]
    return grouped_list

def mean_score_for_feature(a, n_algorithms):
    feat_name, feat_scores = zip(*a)
    feat_name = feat_name[0] # since name is the same for all tuples
    
    m = sum(feat_scores)/float(n_algorithms)
    return (feat_name, m)
    
def keep_top_n(features, n):
    n_algorithms = len(features)
    
    all_feats = []
    for f in features.values():
        all_feats.extend(f)
    
    
    print(all_feats[:8])
    grouped_list = group_by_features(all_feats)
    grouped_list = [mean_score_for_feature(f, n_algorithms) for f in grouped_list]
    
    grouped_list = sorted(grouped_list, key=lambda x: x[1], reverse=True)
    print(grouped_list[:8])
    
    return [x[0] for x in grouped_list[:n]]
    
    
merged_features = keep_top_n(subsets["features"], n=100)
merged_features_lists.append(("Keep Top N features", merged_features, -1))

assert_list_contains_only_unique_features(merged_features)

In [ ]:
def union_of_intersection_two_by_two(features):
    sort_by_len_features = sorted(features.values(), key=lambda x:len(x), reverse=True)
    print([len(f) for f in sort_by_len_features])
    
    def inter(x, y):
        intersection = list(set(x).intersection(set(y)))
        print("Intersection length : %d" % len(intersection))
        return intersection
    
    lists_of_features = [([a[0] for a in f]) for f in sort_by_len_features]
    
    # keep the lists that contains at least 500 features
    lists_of_features = filter(lambda x:len(x) > 500, lists_of_features)
    
    return reduce(inter, lists_of_features)

    
merged_features = union_of_intersection_two_by_two(subsets["features_by_score"])
merged_features_lists.append(("Union of intersections", merged_features, -1))

assert_list_contains_only_unique_features(merged_features)

## Evaluation of the merged subset
Once we have a merged list containing the best features, we would like to evaluate it with several classifiers

_TODO_: use a separate test set ? -> split again train/test set -> no changes in the Dataset class

### Dataset loading
_TODO_: 
* this notebook must only load one dataset
* retrieve dataset to load from cmd arguments or from env variable

In [ ]:
from datasets.EGEOD22619.EGEOD22619Dataset import EGEOD22619Dataset
from datasets.MILE.MileDataset import MileDataset
from datasets.Golub99.GolubDataset import GolubDataset

from datasets.DatasetEncoder import DatasetEncoder
from datasets.DatasetSplitter import DatasetSplitter
from datasets.DatasetLoader import DatasetLoader
from datasets.DatasetBalancer import DatasetBalancer

# Load dataset from environment variable. This is used by automated scripts
ds_class = DatasetLoader.load_from_env_var(default_dataset=DATASET)

print("Dataset used: %s" % ds_class.__name__)

ds = ds_class()

### Dataset transformation
The dataset needs some transformations such as encoding the outputs as float (necessary for scikit learn), normalization, ...

_TODO_:
* dataset splitting (train, test[, validation])
* encode outputs
* normalization
* classes merging
    * due to the low class balancing we might want to regroup them. Example Healthy vs Non-Healthy (choose the most represented class ?)

In [ ]:
# encode Dataset string classes into numbers
ds_encoder = DatasetEncoder(ds)
ds = ds_encoder.encode()

ds = DatasetSplitter(ds, test_size=0.4)

ds_balancer = DatasetBalancer(ds)
ds = ds_balancer.balance()

X = ds.get_X()
y = ds.get_y()

X_train = ds.get_X_train()
y_train = ds.get_y_train()
X_test = ds.get_X_test()
y_test = ds.get_y_test()

class_names = range(len(set(ds.get_y())))

N_FEATURES = len(X_train[0])
print("Number of genes: %d" % N_FEATURES)
print("Dataset samples: %d" % len(y))
print("Train set size %d" % len(X_train))
print("Test set size %d" % len(X_test))

### Assess merged features

TODO: compare performance against a list of random features

### Confusion Matrix

In [ ]:
def show_confusion_matrix(assessment_scores):
    import math

    plt.figure(figsize=(12, 8))

    n_subplots = len(assessment_scores)
    cols = 3
    rows = int(math.ceil(n_subplots / cols))
    i = 1

    for name, score_cm in assessment_scores.iteritems():
        y_test, y_pred = score_cm[1]
        cnf_matrix = confusion_matrix(y_test, y_pred)

        plt.subplot(rows, cols, i)
        i += 1

        ConfusionMatrix.plot(cnf_matrix, classes=class_names,
                              title='Confusion matrix for %s' % name)

    plt.tight_layout()
    plt.show()

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier


def assess_merged_features(clf, clf_name, selected_features, assessment_scores, verbose=True):
    clf.fit(ds.get_X_train(), ds.get_y_train())
    y_pred = clf.predict(ds.get_X_test())
    y_test = ds.get_y_test()

    scores = cross_val_score(clf, ds.get_X_test()[:, selected_features], y_test, cv=5, n_jobs=-1)
    score = np.mean(scores)

    if verbose:
        print("[%s] Score using the merged list of features: %.3f" % (clf_name, score))

    assessment_scores[clf_name] = score, (y_test, y_pred)


def assess_features(selected_features, verbose=True):
    assessment_scores  = {}

    clf = KNeighborsClassifier(algorithm="ball_tree", n_neighbors=5, n_jobs=-1, metric="manhattan")
    assess_merged_features(clf, "KNN", selected_features, assessment_scores, verbose)

    clf = MLPClassifier(solver="adam", alpha=1e-3, hidden_layer_sizes=(100, 50), activation="relu")
    assess_merged_features(clf, "MLP", selected_features, assessment_scores, verbose)

    clf = ExtraTreesClassifier(n_jobs=-1, n_estimators=100)
    assess_merged_features(clf, "ExtraTrees", selected_features, assessment_scores, verbose)
    
    if verbose:
        show_confusion_matrix(assessment_scores)
    
    # mean score
    score = np.median([ass[0] for ass in assessment_scores.values()])
    std = np.std([ass[0] for ass in assessment_scores.values()])
    return score, std

### Merging techniques score

In [ ]:
score_index = 2

for i, m_list in enumerate(merged_features_lists):
    merging_technique, selected_features, score = m_list
    selected_features = list(selected_features)
    score, std = assess_features(selected_features)
    print("---> [%s] median score: %.3f" % (merging_technique, score))

    merged_features_lists[i] = (merging_technique, selected_features, score, std)
    
    # add space for readability
    print("")
    print("")


#### Compare the merged techniques against k random features and against all the features

In [ ]:
# random features
import random

score = 0
std = 0
N = 7
k = 100 # length of the random lists
for _ in range(N):
    random_features = random.sample(range(N_FEATURES), k)
    score_tmp, std_tmp = assess_features(random_features, verbose=False)
    score += score_tmp
    std += std_tmp

score = score/float(N)
std = std/float(N)
print("---> Random features scores: %.3f" % score)

merged_features_lists.append(("%d random features" % k, random_features, score, std))

In [ ]:
# all features

all_features = range(N_FEATURES)
score, std = assess_features(random_features, verbose=True)

print("---> Using all features scores: %.3f" % score)

merged_features_lists.append(("All features", all_features, score, std))

### Plot a bar chart with the mean score for the merging methods

In [ ]:
merging_techniques, merging_lists, scores, stds = zip(*sorted(merged_features_lists, key=lambda x : x[score_index], reverse=True))

y_pos = np.arange(len(merging_techniques))

fig = plt.figure(figsize=(10,4))
ax = fig.add_subplot(111)

ax.bar(y_pos, scores, align='center', yerr=stds, 
       alpha=0.5, width=0.3, color="turquoise", edgecolor="turquoise", ecolor="salmon")

merging_techniques_labels = ["%s\n(#%d)" % (m_name, len(m_list)) for m_name, m_list in zip(merging_techniques, merging_lists)]
plt.xticks(y_pos, merging_techniques_labels)

# add values above the bars
for a,b in enumerate(scores):
    plt.text(a, b, " %.3f" % b, ha='left', va='bottom')

plt.ylabel('Score')
plt.ylim(0.0, 1.1)
plt.title('Median score between several merging methods')
plt.gca().yaxis.grid(True)
plt.tight_layout()
 
plt.show()